In [1]:
# import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')

from sklearn.datasets import fetch_20newsgroups
from nltk.tokenize import word_tokenize #Used to extract words from documents
# from nltk.stem import WordNetLemmatizer #Used to lemmatize words
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans

import sys
from time import time

import pandas as pd
import numpy as np

import spacy
nlp = spacy.load('en_core_web_trf') 

C:\Users\agnes\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
categories = [
    "alt.atheism",
    "comp.graphics",
    "comp.sys.ibm.pc.hardware",
    "comp.sys.mac.hardware",
    "comp.windows.x",
    "misc.forsale",
    "rec.autos",
    "rec.motorcycles",
    "rec.sport.baseball",
    "rec.sport.hockey",
    "sci.crypt",
    "sci.electronics",
    "sci.med",
    "sci.space",
    "soc.religion.christian",
    "talk.politics.guns",
    "talk.politics.mideast",
    "talk.politics.misc",
    "talk.religion.misc",
    "comp.os.ms-windows.misc",
]
data = fetch_20newsgroups(categories=categories)
y_true = data.target
# y_true = newsgroups.targetdata = fetch_20newsgroups(subset='train', categories=['alt.atheism', 'sci.med', 'sci.space'], 
#                              shuffle=False, remove=('headers', 'footers', 'quotes'))

# df = pd.DataFrame(data=data.data)

# len(df)
# print(type(df))
# # df.data[0]
# df.head(3)

In [6]:
df = pd.DataFrame()
df['text'] = data.data


KeyError: 'target'

In [15]:
df['target'] = [categories[x] for x in data.target]
df.head()
print(df['target'][1], df['text'][1], sep="\n")

comp.windows.x
From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>



In [102]:
df['text'][1]

"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

In [103]:
labels = df.target
true_k = len(np.unique(labels)) ## This should be 3 in this example
print(true_k)

20


In [104]:
df_a = df[df['target'] == 'alt.atheism'][:50]
df_b = df[df['target'] == 'rec.motorcycles'][:50]
# df_c = df[df['target'] == 'comp.sys.ibm.pc.hardware']
df = pd.concat([df_a, df_b])
df.reset_index(drop=True, inplace=True)
df

,text,target
0,\nDon't be so sure. Look what happened to Jap...,alt.atheism
1,"\n[...]\n\nThese don't seem like ""little thing...",alt.atheism
2,"[deletions...]\n\n\n\nFirst of all, infinity i...",alt.atheism
3,".d.\n.d.\nI just thought of another one, in th...",alt.atheism
4,"\nYes, this is also my understanding of the ma...",alt.atheism
...,...,...
95,,rec.motorcycles
96,"\n\nhmmmm.. not sure, since no such beast exis...",rec.motorcycles
97,\n[stuff about autobahn and safety of sho at s...,rec.motorcycles
98,\n-> I saw an interesting product in NY Auto S...,rec.motorcycles


In [105]:
# Токенизация
def number_normalizer(tokens):
    """Map all numeric tokens to a placeholder.

    For many applications, tokens that begin with a number are not directly
    useful, but the fact that such a token exists can be relevant.  By applying
    this form of dimensionality reduction, some methods may perform better.
    """
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)

for i in range(len(df['text'])):
    tokenized_text = word_tokenize(df['text'][i])
    # tokenized_text = number_normalizer(tokenized_text)
    tokenized_text = [' '.join([w for w in tokenized_text if w.isalpha() and len(w)>1])]
    df['text'][i] = ' '.join([f.lower() for f in tokenized_text])
    
df

,text,target
0,do be so sure look what happened to japanese c...,alt.atheism
1,these do seem like little things to me at leas...,alt.atheism
2,deletions first of all infinity is mathematica...,alt.atheism
3,just thought of another one in the bible so it...,alt.atheism
4,yes this is also my understanding of the major...,alt.atheism
...,...,...
95,,rec.motorcycles
96,hmmmm not sure since no such beast exists can ...,rec.motorcycles
97,stuff about autobahn and safety of sho at spee...,rec.motorcycles
98,saw an interesting product in ny auto show and...,rec.motorcycles


In [106]:
def remove_names(text):
    doc = nlp(text)
    newString = text
    for e in reversed(doc.ents):
        if e.label_ == "PERSON": # Only if the entity is a PERSON
            newString = newString[:e.start_char] + newString[e.start_char + len(e.text):]
    return newString


for i in range(len(df['text'])):
    df['text'][i] = remove_names(df['text'][i])
df

,text,target
0,do be so sure look what happened to japanese c...,alt.atheism
1,these do seem like little things to me at leas...,alt.atheism
2,deletions first of all infinity is mathematica...,alt.atheism
3,just thought of another one in the bible so it...,alt.atheism
4,yes this is also my understanding of the major...,alt.atheism
...,...,...
95,,rec.motorcycles
96,hmmmm not sure since no such beast exists can ...,rec.motorcycles
97,stuff about autobahn and safety of sho at spee...,rec.motorcycles
98,saw an interesting product in ny auto show and...,rec.motorcycles


In [94]:
df['text'][7]

'can somebody reconcile the apparent contradiction between and'

In [107]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

for i in range(len(df['text'])):
    df['text'][i] = [word for word in df['text'][i].split() if word not in stop_words]
    df['text'][i] = " ".join(str(x) for x in df['text'][i])

df.to_csv('preprocessed_docs.csv', index=False)

df

,text,target
0,sure look happened japanese citizens us world ...,alt.atheism
1,seem like little things least orders worse mot...,alt.atheism
2,deletions first infinity mathematical concept ...,alt.atheism
3,thought another one bible definately lack reli...,alt.atheism
4,yes also understanding majority islamic laws h...,alt.atheism
...,...,...
95,,rec.motorcycles
96,hmmmm sure since beast exists tell another tho...,rec.motorcycles
97,stuff autobahn safety sho speed deleted mustan...,rec.motorcycles
98,saw interesting product ny auto show would lik...,rec.motorcycles


In [96]:
for i in range(len(df['text'])):
    df['text'][i] = df['text'][i].split()
df

,text,target
0,"[sure, look, happened, japanese, citizens, us,...",alt.atheism
1,"[seem, like, little, things, least, orders, wo...",alt.atheism
2,"[deletions, first, infinity, mathematical, con...",alt.atheism
3,"[thought, another, one, bible, definately, lac...",alt.atheism
4,"[yes, also, understanding, majority, islamic, ...",alt.atheism
...,...,...
95,[],rec.motorcycles
96,"[hmmmm, sure, since, beast, exists, tell, anot...",rec.motorcycles
97,"[stuff, autobahn, safety, sho, speed, deleted,...",rec.motorcycles
98,"[saw, interesting, product, ny, auto, show, wo...",rec.motorcycles


In [108]:
df = df[df['text'].apply(lambda l: len(l) > 0)]
df.shape

(94, 2)

In [35]:
%%time
# обучаем Word2Vec на наших данных
import multiprocessing
from gensim.models import Word2Vec


cores = multiprocessing.cpu_count()
print(f'Ядер: {cores}')

model = Word2Vec(df['text'],
                  min_count=5,
                  window=5,
                  vector_size=100,
                  sample=1e-3,
                  alpha=0.025,
                  min_alpha=0.0001,
                  negative=5,
                  workers=cores-1,
                  epochs=5)

print(f'Размер словаря: {len(model.wv.vectors)}')
print(f'Словарь: {model.wv.key_to_index}')

Ядер: 8
Размер словаря: 4495
Словарь: {'max': 0, 'one': 1, 'would': 2, 'windows': 3, 'file': 4, 'use': 5, 'image': 6, 'also': 7, 'like': 8, 'know': 9, 'bhj': 10, 'giz': 11, 'get': 12, 'people': 13, 'files': 14, 'think': 15, 'god': 16, 'program': 17, 'system': 18, 'could': 19, 'using': 20, 'many': 21, 'anyone': 22, 'well': 23, 'graphics': 24, 'problem': 25, 'time': 26, 'good': 27, 'see': 28, 'version': 29, 'even': 30, 'software': 31, 'may': 32, 'need': 33, 'thanks': 34, 'available': 35, 'much': 36, 'wm': 37, 'data': 38, 'please': 39, 'used': 40, 'find': 41, 'way': 42, 'say': 43, 'jpeg': 44, 'something': 45, 'make': 46, 'ah': 47, 'new': 48, 'dos': 49, 'information': 50, 'ca': 51, 'help': 52, 'must': 53, 'want': 54, 'images': 55, 'gk': 56, 'work': 57, 'card': 58, 'believe': 59, 'first': 60, 'read': 61, 'etc': 62, 'us': 63, 'point': 64, 'two': 65, 'since': 66, 'ftp': 67, 'format': 68, 'set': 69, 'true': 70, 'computer': 71, 'try': 72, 'without': 73, 'said': 74, 'bj': 75, 'support': 76, 'fre

In [15]:
# texts_as_wordlists = [x.split(' ') for x in df['text']]
# # print(texts_as_wordlists)
# text_vectors = [model.wv.get_mean_vector(wordlist) for wordlist in texts_as_wordlists]
# print(text_vectors)

In [36]:
df_a = df[df['target'] == 'alt.atheism'][:50]
df_b = df[df['target'] == 'comp.graphics'][:50]
df_c = df[df['target'] == 'comp.sys.ibm.pc.hardware'][:50]
df = pd.concat([df_a, df_b, df_c])

In [37]:
df

,text,target
0,"[sure, look, happened, japanese, citizens, us,...",alt.atheism
1,"[seem, like, little, things, least, orders, wo...",alt.atheism
2,"[deletions, first, infinity, mathematical, con...",alt.atheism
3,"[thought, another, one, bible, definately, lac...",alt.atheism
4,"[yes, also, understanding, majority, islamic, ...",alt.atheism
...,...,...
1110,"[might, problem, video, monitor, instead, many...",comp.sys.ibm.pc.hardware
1111,"[searching, phonetic, truetype, font, windows,...",comp.sys.ibm.pc.hardware
1113,"[official, request, discussion, rfd, creation,...",comp.sys.ibm.pc.hardware
1114,"[nothing, beats, skiing, want, real, fun, holi...",comp.sys.ibm.pc.hardware


In [38]:
X = []
map = {}
df['vectorized'] = ""

for (i, tokens) in df['text'].iteritems():
    vecs = []
    for token in tokens:
        # print(token)
        if model.wv.has_index_for(token):
            vecs.append(model.wv.get_vector(token))
    if vecs:
        vec_avg = np.asarray(vecs).mean(axis=0)
        X.append(vec_avg)
        df['vectorized'][i] = vec_avg
        map[len(X) - 1] = i
    else:
        print(i, tokens)

print(f'Векторов предложений: {len(X)}')


Векторов предложений: 150


In [40]:
# df['text'][7]
# X
# df = []
# df.head(8)
df

,text,target,vectorized
0,"[sure, look, happened, japanese, citizens, us,...",alt.atheism,"[-0.35489327, 0.46809992, -0.076527625, 0.2263..."
1,"[seem, like, little, things, least, orders, wo...",alt.atheism,"[-0.48077837, 0.6430152, -0.12509969, 0.292336..."
2,"[deletions, first, infinity, mathematical, con...",alt.atheism,"[-0.43910798, 0.5831662, -0.11837247, 0.265508..."
3,"[thought, another, one, bible, definately, lac...",alt.atheism,"[-0.40613893, 0.53376985, -0.086722, 0.2599461..."
4,"[yes, also, understanding, majority, islamic, ...",alt.atheism,"[-0.31977788, 0.4211217, -0.061175305, 0.20968..."
...,...,...,...
1110,"[might, problem, video, monitor, instead, many...",comp.sys.ibm.pc.hardware,"[-0.36224288, 0.48566166, -0.088247694, 0.2165..."
1111,"[searching, phonetic, truetype, font, windows,...",comp.sys.ibm.pc.hardware,"[-0.56978196, 0.7661294, -0.19079222, 0.311557..."
1113,"[official, request, discussion, rfd, creation,...",comp.sys.ibm.pc.hardware,"[-0.45096523, 0.6047655, -0.11184164, 0.260900..."
1114,"[nothing, beats, skiing, want, real, fun, holi...",comp.sys.ibm.pc.hardware,"[-0.32326558, 0.42236632, -0.05138647, 0.21445..."


In [39]:
df = df[df['vectorized'].apply(lambda l: len(l) > 0)]
df.shape

(150, 3)

In [42]:
r = [] 
for (i, tokens) in df['vectorized'].iteritems():
    r.append(tokens)

df2 = pd.DataFrame(r)
df2.index = df.index
df2['target']=df['target']


df2.to_csv('files\word2vec3by50.csv', index=True)
df.to_csv('files\preprocessed3by50.csv')
df2.head(8)


,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,target
0,-0.354893,0.468100,-0.076528,0.226385,0.247830,0.040653,-0.146507,0.331224,-0.248520,0.396161,...,0.193366,0.192567,0.025828,0.025863,0.331648,0.000792,-0.210656,-0.039354,0.097044,alt.atheism
1,-0.480778,0.643015,-0.125100,0.292337,0.345799,0.108446,-0.210583,0.333875,-0.296069,0.551135,...,0.258347,0.229829,-0.004990,-0.029753,0.433619,0.018759,-0.227618,-0.051830,0.106234,alt.atheism
2,-0.439108,0.583166,-0.118372,0.265508,0.312333,0.096959,-0.190046,0.304370,-0.265254,0.500772,...,0.236195,0.210110,-0.004490,-0.028976,0.387291,0.011695,-0.207815,-0.052112,0.094935,alt.atheism
3,-0.406139,0.533770,-0.086722,0.259946,0.283312,0.053283,-0.170889,0.374653,-0.278275,0.455121,...,0.219207,0.221063,0.026541,0.025911,0.383689,-0.001525,-0.238711,-0.043844,0.111409,alt.atheism
4,-0.319778,0.421122,-0.061175,0.209681,0.221164,0.026492,-0.128943,0.327613,-0.231931,0.355741,...,0.174303,0.181131,0.030239,0.039840,0.306753,-0.004552,-0.203132,-0.032391,0.092678,alt.atheism
5,-0.375785,0.498474,-0.075357,0.247697,0.258709,0.038594,-0.158257,0.366723,-0.269246,0.418308,...,0.203368,0.207862,0.030153,0.035733,0.357706,-0.005003,-0.231217,-0.040945,0.109710,alt.atheism
6,-0.332732,0.442254,-0.077051,0.213667,0.235627,0.042810,-0.139075,0.305981,-0.228811,0.374757,...,0.182947,0.181086,0.019388,0.019471,0.315539,-0.000781,-0.192693,-0.036208,0.089095,alt.atheism
7,-0.131525,0.173758,-0.019931,0.093283,0.086069,-0.005545,-0.045693,0.168418,-0.103246,0.137400,...,0.071046,0.086841,0.025087,0.038847,0.128616,-0.004796,-0.098400,-0.008775,0.051482,alt.atheism


In [43]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=20, random_state=42)
X_svd = svd.fit_transform(X)

print(X_svd.shape)

r = [] 
for elem in X_svd:
    r.append(elem)

df3 = pd.DataFrame(r)
df3.index = df.index

df3['target'] = df['target']
df3.head(8)

df3.to_csv('files\svd_word2vec3by50.csv', index=True)
# df.to_csv('files\preprocessed3by50.csv')



(150, 20)


In [129]:
for i in range(len(df['text'])):
    df['text'][i] = " ".join(df['text'][i])
df

C:\Users\agnes\AppData\Local\Temp\ipykernel_4492\1982565736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i] = " ".join(df['text'][i])


KeyError: 7

In [62]:
df_a = df[df['target'] == 'alt.atheism'][:50]
df_b = df[df['target'] == 'rec.motorcycles'][:50]
df = pd.concat([df_a, df_b])
# print(df.shape)

(100, 2)


In [115]:
vectorizer = TfidfVectorizer(strip_accents='unicode') ## Corpus is in English
X_TFIDF = vectorizer.fit_transform(df['text'])
X_TFIDF

<94x3032 sparse matrix of type '<class 'numpy.float64'>'
	with 5627 stored elements in Compressed Sparse Row format>

In [121]:
print(X_TFIDF.shape)
print(type(X_TFIDF))
# for i in range(len(X_TFIDF[0])):
#     print(*X_TFIDF[i, :].toarray())
svd = TruncatedSVD(n_components=20, random_state=42)
X_svd = svd.fit_transform(X_TFIDF)

print(X_svd.shape)
X_svd

(94, 3032)
<class 'scipy.sparse._csr.csr_matrix'>
(94, 20)


array([[ 0.13559643,  0.01651618, -0.03670574, ..., -0.0587421 ,
         0.24965846,  0.09938825],
       [ 0.16783712,  0.04660422, -0.15006388, ...,  0.0577376 ,
        -0.13161992,  0.05417895],
       [ 0.18900953, -0.01933397, -0.01934694, ..., -0.02157542,
        -0.03109779,  0.08499356],
       ...,
       [ 0.27526083, -0.15187362,  0.03363173, ..., -0.19863838,
         0.02260139, -0.05966691],
       [ 0.15921202, -0.07765307,  0.00535431, ...,  0.00974303,
         0.13663073,  0.00415655],
       [ 0.09578921, -0.04498711, -0.05021247, ...,  0.12342038,
        -0.05449813,  0.09441884]])

In [116]:
df3 = pd.DataFrame(X_TFIDF.toarray())
df3.index = df.index
df3['target'] = df['target']
df3

,0,1,2,3,4,5,6,7,8,9,...,3023,3024,3025,3026,3027,3028,3029,3030,3031,target
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,alt.atheism
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,alt.atheism
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,alt.atheism
3,0.0,0.0,0.0,0.0,0.059566,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,alt.atheism
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.033353,0.00000,0.0,0.0,0.0,0.0,alt.atheism
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,rec.motorcycles
96,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,rec.motorcycles
97,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.083986,0.00000,0.0,0.0,0.0,0.0,rec.motorcycles
98,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.12601,0.0,0.0,0.0,0.0,rec.motorcycles


In [117]:
res = pd.DataFrame(df3)
res.to_csv('files/svd_TFIDF2by50.csv', index=True)
res

,0,1,2,3,4,5,6,7,8,9,...,3023,3024,3025,3026,3027,3028,3029,3030,3031,target
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,alt.atheism
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,alt.atheism
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,alt.atheism
3,0.0,0.0,0.0,0.0,0.059566,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,alt.atheism
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.033353,0.00000,0.0,0.0,0.0,0.0,alt.atheism
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,rec.motorcycles
96,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,rec.motorcycles
97,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.083986,0.00000,0.0,0.0,0.0,0.0,rec.motorcycles
98,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.12601,0.0,0.0,0.0,0.0,rec.motorcycles
